In [1]:
import helper_func
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [2]:
from pyspark.sql.functions import desc
import folium


In [3]:
node = sqlContext.read.load("Data/hungary.osm.pbf.node.parquet")
way = sqlContext.read.load("Data/hungary.osm.pbf.way.parquet")
relations = sqlContext.read.load("Data/hungary.osm.pbf.relation.parquet")

In [4]:
node.printSchema()

root
 |-- id: long (nullable = true)
 |-- version: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- changeset: long (nullable = true)
 |-- uid: integer (nullable = true)
 |-- user_sid: binary (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: binary (nullable = true)
 |    |    |-- value: binary (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)



In [5]:
f1 = helper_func.filter_string_match('highway')
f2 = helper_func.make_column_tag_value_key('highway')

filtered_way = way.select('*').filter(f1(way.tags))
extracted_highway = filtered_way.withColumn('extract_tag',f2(filtered_way.tags))

yo


In [6]:
only_tags = extracted_highway.select('id','extract_tag')

In [7]:
only_tags.show()

+-------+-----------+
|     id|extract_tag|
+-------+-----------+
|3175810|residential|
|3175943|residential|
|3175983|residential|
|3192356|  secondary|
|3212111|  secondary|
|3390680|    primary|
|3414164|   tertiary|
|3425635|  secondary|
|3426019|  secondary|
|3426020|residential|
|3426236|  secondary|
|3427029|  secondary|
|3427031|  secondary|
|3427032|  secondary|
|3427034|  secondary|
|3427036|  secondary|
|3427783|    primary|
|3427999|residential|
|3428708|residential|
|3428709|   tertiary|
+-------+-----------+
only showing top 20 rows



In [8]:
only_tags.groupBy("extract_tag").count().sort(desc("count")).show()


+-------------+------+
|  extract_tag| count|
+-------------+------+
|        track|240048|
|  residential|167600|
|      service|108138|
|      footway| 63573|
|         path| 43916|
|    secondary| 26914|
|      primary| 21218|
| unclassified| 19533|
|     tertiary| 18214|
|        steps|  8554|
|     cycleway|  6585|
|     motorway|  3416|
|living_street|  3261|
|   pedestrian|  2762|
|motorway_link|  2175|
|     platform|  1593|
|        trunk|  1240|
| primary_link|  1179|
| construction|   984|
|   trunk_link|   881|
+-------------+------+
only showing top 20 rows



In [9]:
f1 = helper_func.filter_string_match('taxi')
f2 = helper_func.make_column_tag_value_key('amenity')
filtered_node = node.select('id','latitude','longitude','tags').filter(f1(node.tags))

yo


In [10]:
taxi_df = filtered_node.collect()

In [11]:
taxi_coods = helper_func.get_lat_long(taxi_df)

In [12]:
m = folium.Map(
    world_copy_jump=False,location=[47.5145367, 19.0385427],
    no_wrap=False,zoom_start=7)


In [50]:
def add_markers(m,list_of_lat_and_long_and_ID):
    i = 0
    for x in list_of_lat_and_long_and_ID:
        if i < 100:
            folium.Marker(location=[x[1],x[2]],popup=str(x[0])+','+str(x[1])+','+str(x[2]),size = 0.1).add_to(m)
            i+=1
        else:
            break
add_markers(m,taxi_coods)

In [51]:
m

In [52]:
f1 = helper_func.filter_string_match('amenity')
f2 = helper_func.make_column_tag_value_key('amenity')
f3 = helper_func.make_column_tag_value_key('name')
 
filtered_node = node.select('id','latitude','longitude','tags').filter(f1(node.tags))
extracted_amenities = filtered_node.withColumn('extract_tag',f2(filtered_node.tags))
extracted_amenities = extracted_amenities.withColumn('extract_name',f3(extracted_amenities.tags))

yo
yo


In [41]:
amenities_df = extracted_amenities.collect()

In [44]:
# extracted_amenities.groupBy("extract_tag").count().sort(desc("count")).show()
extracted_amenities.printSchema()

root
 |-- id: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- key: binary (nullable = true)
 |    |    |-- value: binary (nullable = true)
 |-- extract_tag: string (nullable = true)
 |-- extract_name: string (nullable = true)



In [18]:
# extracted_amenities.groupBy("extract_tag").count().sort(desc("count")).show()

In [45]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import FloatSlider
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
# interact_manual(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5));


In [46]:
def filter_amenitites(amenities_df,string):
    list1 =[]
    for x in amenities_df:
        if x.extract_tag == string:
            list1.append(x)
    return list1
        

In [64]:
def add_markers2(m,list_of_lat_and_long_and_ID):
    i = 0
    for x in list_of_lat_and_long_and_ID:
        if i < 100:
            folium.Marker(location=[x[1],x[2]],popup=str(x[3]),size = 0.1).add_to(m)
            i+=1
        else:
            break


In [65]:
m1 = folium.Map(
    world_copy_jump=False,location=[47.5145367, 19.0385427],
    no_wrap=False,zoom_start=7)

def get_lat_long(df):
    list1 = []
    for x in df:
        list1.append([x.id,x.latitude , x.longitude,x.extract_name]) 
    return list1     

In [69]:
@interact
def show_articles_more_than(column='drinking_water'):
    filtered =  filter_amenitites(amenities_df,column)
    filtered_coods = get_lat_long(filtered)
    m1 = folium.Map(world_copy_jump=False,location=[47.5145367, 19.0385427],no_wrap=False,zoom_start=7)
    add_markers2(m1,filtered_coods[:100])
    return m1


# YOU CAN CHANGE THE AMENITY NAME HERE AND SEE WHERE IT LIES ON THE MAP

interactive(children=(Text(value='drinking_water', description='column'), Output()), _dom_classes=('widget-int…

In [70]:
amenities_df

[Row(id=17585702, latitude=46.8886574, longitude=17.8917705, tags=[Row(key=bytearray(b'amenity'), value=bytearray(b'ferry_terminal')), Row(key=bytearray(b'ferry'), value=bytearray(b'yes')), Row(key=bytearray(b'name'), value=bytearray(b'Tihanyr\xc3\xa9v')), Row(key=bytearray(b'public_transport'), value=bytearray(b'stop_position'))], extract_tag='ferry_terminal', extract_name='Tihanyrév'),
 Row(id=29634738, latitude=47.766896100000004, longitude=18.1247519, tags=[Row(key=bytearray(b'amenity'), value=bytearray(b'place_of_worship')), Row(key=bytearray(b'denomination'), value=bytearray(b'baptist')), Row(key=bytearray(b'religion'), value=bytearray(b'christian'))], extract_tag='place_of_worship', extract_name='[B@4f64bd'),
 Row(id=30761500, latitude=46.8841216, longitude=17.748458600000003, tags=[Row(key=bytearray(b'addr:city'), value=bytearray(b'Balatonakali')), Row(key=bytearray(b'addr:country'), value=bytearray(b'HU')), Row(key=bytearray(b'addr:housenumber'), value=bytearray(b'26')), Row(k

In [74]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import size
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf 
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.types import *
import folium
from pyspark.sql.functions import desc



def make_column_tag_value_key(string1):
    string = string1
    print('yo')
    def tag_extractor_helper(x):
        number_of_tags = len(x)
        for i in range(0,number_of_tags):
            print(type(i))
            if (string in x[i][0].decode("latin1")):
                return x[i][1].decode("latin1")
        return x[0][1]

    tag_extractor_udf = udf(tag_extractor_helper, StringType())
    return tag_extractor_udf


f1 = helper_func.filter_string_match('amenity')
f2 = make_column_tag_value_key('amenity')
f3 = make_column_tag_value_key('name')
 
filtered_node = node.select('id','latitude','longitude','tags').filter(f1(node.tags))
extracted_amenities = filtered_node.withColumn('extract_tag',f2(filtered_node.tags))
extracted_amenities = extracted_amenities.withColumn('extract_name',f3(extracted_amenities.tags))

yo
yo
